In [1]:
#!conda list

In [2]:
import pygrib
import pandas as pd
import numpy as np

In [3]:
gribName="20151018_1024_9b49a5d5"
cyc_name=str.upper("Patricia")
grbs = pygrib.open('.\\Data\\'+gribName+'.grib')

In [4]:
grb=grbs[1:]
grb.append(grbs[len(grb)+1])

In [5]:
#selected_grbs=grbs.select(shortName="u",typeOfLevel='isobaricInhPa',level=[500,700,800,925,1000])
#selected_grbs[0]

In [5]:
numChannels=12
gridShape=(221,381)
framesPerDay=24
isoLvls=[500,700,850,925,1000]
numIsoLvls=len(isoLvls)
numFrames=len(grb)
numDays=numFrames//(numChannels*framesPerDay*numIsoLvls)
#INDEX=Frame*5*12+IsoLvl*12+Channel (tot=12*5*24*numDays)
assert numFrames%(numChannels*framesPerDay*numIsoLvls)==0 #Having integer number of days
assert grb[1].values.shape==gridShape #Check grid shape matches

In [6]:
#firstFrame=grb[0]['day']

In [8]:
#grb[0].keys()

In [9]:
#grb[0:13]

In [7]:
arr=np.zeros(shape=(numDays*framesPerDay,numIsoLvls,numChannels,*gridShape),dtype="float32")
print(arr.shape)

def parseIndices(i):
    timeLoc=i//(numChannels*numIsoLvls)
    isoLoc=(i-timeLoc*(numChannels*numIsoLvls))//numChannels
    channelLoc=i-timeLoc*(numChannels*numIsoLvls)-isoLoc*numChannels
    return timeLoc,isoLoc,channelLoc

#INDEX=Frame*60+IsoLvl*12+Channel (tot=12*5*24*numDays)
for i,v in enumerate(grb):
    arr[*parseIndices(i)] = v.values.astype("float32")


(168, 5, 12, 221, 381)


In [8]:
parseIndices(247)

(4, 0, 7)

In [12]:
#grb[247].values

In [13]:
del grb
del grbs

In [14]:
#arr[0,0,1]

In [9]:
import numpy as np

def find_ref_minmax(data):
    assert data.shape[2]==60 #60 channels
    minarr=np.zeros(60)
    maxarr=np.zeros(60)
    for channel in range(data.shape[2]):  # Assuming the third index is for channels
        # Extract the channel data
        channel_data = data[:, :, channel, :, :]
        
        # Compute the min and max
        minarr[channel] = np.min(channel_data)
        maxarr[channel] = np.max(channel_data)
    return minarr,maxarr

# Assuming `arr` is your ndarray with the shape (numDays*framesPerDay, numIsoLvls, numChannels, *gridShape)
def normalize_channels(data,minarr,maxarr):
    # data.shape should be (numDays*framesPerDay, numIsoLvls, numChannels, height, width)
    assert data.shape[2]==60 #60 channels
    # Initialize a new array to hold the normalized data
    normalized_data = np.zeros_like(data, dtype=np.float32)
    
    # Iterate over each channel to normalize
    for channel in range(data.shape[2]):  # Assuming the third index is for channels
        # Extract the channel data
        channel_data = data[:, :, channel, :, :]
        
        # OBTAIN the min and max
        min_val = minarr[channel]
        max_val = maxarr[channel]
        
        # Avoid division by zero in case all values in the channel are the same
        if max_val > min_val:
            # Normalize the channel
            normalized_data[:, :, channel, :, :] = (channel_data - min_val) / (max_val - min_val)
        else:
            # If max equals min, set the normalized data to zero or handle appropriately
            normalized_data[:, :, channel, :, :] = 0  # Or set it to a neutral value like 0.5 if more appropriate
    return normalized_data

minref,maxref=find_ref_minmax(arr)
# Apply the normalization to your data
normalized_arr = normalize_channels(arr,minref,maxref)

In [16]:
import numpy as np

# Assuming normalized_arr is your ndarray
# Example shape of normalized_arr is (numDays*framesPerDay, numIsoLvls, numChannels, height, width)
# We need to transform it to (numDays*framesPerDay, height, width, numChannels*numIsoLvls)

# First, let's transpose the axes to bring all spatial dimensions next to each other
# Transpose (0, 3, 4, 1, 2) means:
# 0 -> 0 (keep numDays*framesPerDay at the first axis)
# 1 -> 3 (move the first spatial dimension to second place)
# 2 -> 4 (move the second spatial dimension to third place)
# 3 -> 1 (move numIsoLvls to the last but one)
# 4 -> 2 (move numChannels to the last position)
transposed_arr = np.transpose(normalized_arr, (0, 3, 4, 1, 2))

# Now, we need to merge the last two dimensions (numIsoLvls and numChannels)
# Calculate the new dimension
new_last_dim = transposed_arr.shape[3] * transposed_arr.shape[4]

# Reshape to merge the last two dimensions
inTensor = transposed_arr.reshape(transposed_arr.shape[0], transposed_arr.shape[1], transposed_arr.shape[2], new_last_dim)

# reshaped_arr now has the shape (numDays*framesPerDay, height, width, numChannels*numIsoLvls)

In [17]:
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
# from tensorflow.keras.models import Model

# # Assuming each "frame" is an independent input not wrapped in TimeDistributed
# input_layer = Input(shape=(13, *gridShape, 60))  # Adjust based on your exact gridShape and channel info

# # CNN layers applied independently to each frame (you might need a loop or custom layer here)
# # Simplified version: Applying CNN to the first frame for illustration (adjust this part!)
# cnn_out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(input_layer[:,0,:,:,:])
# cnn_out = MaxPooling2D(pool_size=(3, 3))(cnn_out)
# cnn_out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(cnn_out)
# cnn_out = MaxPooling2D(pool_size=(3, 3))(cnn_out)
# cnn_out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(cnn_out)
# cnn_out = MaxPooling2D(pool_size=(2, 2))(cnn_out)
# cnn_out = Flatten()(cnn_out)

# # Scalar inputs for the best track data at present (shape=(6,))
# best_track_input = Input(shape=(6,))

# # Concatenate the scalar values with the output of the CNN
# combined_inputs = Concatenate(axis=-1)([cnn_out, best_track_input])

# # Dense layers for final prediction
# dense_out = Dense(units=100, activation='relu')(combined_inputs)
# output = Dense(units=4)(dense_out)  # Predicting values for the cyclone's next location

# # Compile the model
# model = Model(inputs=[input_layer, best_track_input], outputs=output)
# model.compile(optimizer='adam', loss='mean_squared_error')

# model.summary()

In [1]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model

# Input shape considering each frame is now part of a sequence
input_layer = Input(shape=(13, *gridShape, 60))  # Adjust the gridShape and channels as necessary

# Apply CNN to each frame independently using TimeDistributed
cnn_out = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))(input_layer)
cnn_out = TimeDistributed(MaxPooling2D(pool_size=(3, 3)))(cnn_out)
cnn_out = TimeDistributed(Conv2D(filters=64, kernel_size=(4, 4), activation='relu'))(cnn_out)
cnn_out = TimeDistributed(MaxPooling2D(pool_size=(3, 3)))(cnn_out)
cnn_out = TimeDistributed(Conv2D(filters=128, kernel_size=(4, 4), activation='relu'))(cnn_out)
cnn_out = TimeDistributed(MaxPooling2D(pool_size=(3, 3)))(cnn_out)
cnn_out = TimeDistributed(Flatten())(cnn_out)  # Flatten each frame's feature map

# If you want to aggregate features across time, you can use Flatten directly or a pooling layer across the time dimension
# Flatten to prepare for Dense layers (e.g., merging time and feature maps into one long vector)
cnn_out = Flatten()(cnn_out)

# Scalar inputs for the best track data at present (shape=(6,))
best_track_input = Input(shape=(6,))

# Concatenate the scalar values with the output of the CNN
combined_inputs = Concatenate(axis=-1)([cnn_out, best_track_input])

# Dense layers for final prediction
dense_out = Dense(units=100, activation='relu')(combined_inputs)
output = Dense(units=4)(dense_out)  # Predicting 4 values for the cyclone's next location

# Compile the model
model = Model(inputs=[input_layer, best_track_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

NameError: name 'gridShape' is not defined

In [19]:
best_Track=pd.read_csv("BestTrack_Trimmed.csv")

In [20]:
# firstFrame

10

In [21]:
best_Track['ISO_TIME']=pd.to_datetime(best_Track['ISO_TIME'])

In [22]:
starttime=pd.to_datetime(gribName[:8],format='%Y%m%d')
endtime=starttime+pd.Timedelta(days=numDays)

In [23]:
fBesttrack=best_Track[(best_Track['ISO_TIME']>=starttime)&(best_Track['ISO_TIME']<=endtime)&(best_Track['LAT']>=0.0)&(best_Track['LAT']<=90.0)&(best_Track['LON']>=-170.0)&(best_Track['LON']<=-80.0)]

In [24]:
fBesttrack=fBesttrack[fBesttrack['NAME']==cyc_name]

In [25]:
selected_columns = fBesttrack[['USA_WIND', 'USA_PRES', 'LAT', 'LON','ISO_TIME']]
selected_columns=selected_columns[selected_columns['ISO_TIME'].dt.hour%3==0]
def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# Define the ranges
lat_min, lat_max = 0, 40
lon_min, lon_max = -170, -80
wind_min, wind_max = 20, 200
pressure_min, pressure_max = 860, 1020

# Normalize the data
selected_columns['LAT'] = normalize(selected_columns['LAT'], lat_min, lat_max)
selected_columns['LON'] = normalize(selected_columns['LON'], lon_min, lon_max)
selected_columns['USA_WIND'] = normalize(selected_columns['USA_WIND'].astype(float), wind_min, wind_max)
selected_columns['USA_PRES'] = normalize(selected_columns['USA_PRES'].astype(float), pressure_min, pressure_max)
selected_columns['ISO_DATE'] = selected_columns['ISO_TIME'].dt.dayofyear/365
selected_columns['ISO_TIME'] = selected_columns['ISO_TIME'].dt.hour/24

In [26]:
import datetime
starttimeb=fBesttrack['ISO_TIME'].iloc[0]
endtimeb=fBesttrack['ISO_TIME'].iloc[-1]
assert (endtimeb-starttimeb)/datetime.timedelta(seconds=3*3600) == len(selected_columns)-1 #same frame count

besttrack_ndarr=selected_columns.to_numpy().astype("float32")

In [27]:
# import pandas as pd
# df = pd.read_csv('ibtracs.EP.list.v04r00.csv')

In [28]:
# df_trim=df[['SID','NAME','SEASON','NUMBER','NATURE','ISO_TIME','LAT','LON','WMO_WIND','WMO_PRES','USA_RECORD','USA_STATUS','USA_WIND','USA_PRES','USA_GUST']][1:]
# df_trim['SEASON']=pd.to_numeric(df_trim['SEASON'])
# df_trim=df_trim[df_trim['SEASON']>=2015]

In [29]:
#df_trim.to_csv('BestTrack_Trimmed.csv')

In [30]:
#import cdsapi
#c = cdsapi.Client()
#c.retrieve()

In [31]:
(inTensor.size*inTensor.itemsize)/(1024*1024*1024)#in GB

3.1618255376815796

In [32]:
startFrameinA=int((starttimeb-starttime)/datetime.timedelta(hours=1))
endFrameinA=int((endtimeb-starttime)/datetime.timedelta(hours=1))

In [33]:
import numpy as np
import tensorflow as tf

def Bdx_to_Adx(bdx, startFrameinA):
    """ Converts an index in best track data to its corresponding index in inTensor. """
    return bdx * 3 + startFrameinA

def data_generator(inTensor, besttrack_ndarr, startFrameinA, endFrameinA, scope=12, forecast_horizon=6):
    """
    Generator function to yield individual training data samples and labels.
    """
    # Calculate the number of samples available for generation
    num_samples = len(besttrack_ndarr) - forecast_horizon // 3
    indices = np.arange(0, num_samples)
    np.random.shuffle(indices)  # Shuffle indices for random sampling
    
    while True:  # Loop indefinitely for continuous generation
        for idx in indices:
            # Calculate starting frame index for inTensor based on best track data index
            start_idx = Bdx_to_Adx(idx, startFrameinA) - scope
            end_idx = Bdx_to_Adx(idx, startFrameinA) + 1
            
            # Generate input frames from inTensor
            cnn_input = np.stack(inTensor[start_idx:end_idx])
            
            # Get corresponding best track input
            track_input = besttrack_ndarr[idx]
            
            # Prepare target output based on forecast horizon
            target = besttrack_ndarr[idx + forecast_horizon // 3][:4]
            
            if cnn_input is None or track_input is None or target is None:
                print("None found in generator output!")
                continue  # or handle it differently as needed
            
            # print("Yielding data:", cnn_input.shape, track_input.shape, target.shape)
            # Yield the individual data sample and target
            yield ((cnn_input, track_input), target)

# Define output signatures to specify the shapes and types of outputs expected
output_signature = (
    (
        tf.TensorSpec(shape=(13, *gridShape, 60), dtype=tf.float32),  # CNN input shape
        tf.TensorSpec(shape=(6,), dtype=tf.float32)  # Best track input shape
    ),
    tf.TensorSpec(shape=(4,), dtype=tf.float32)  # Output target shape
)

# Create a TensorFlow dataset from the generator, specifying batch size within the dataset pipeline
dataset = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(inTensor, besttrack_ndarr, startFrameinA, endFrameinA),
    output_signature=output_signature
).batch(1)  # Define batch size here

# Example usage: Ready for model training
# model.fit(dataset, epochs=10)

In [34]:
try:
    model.fit(x=dataset, steps_per_epoch=4, epochs=5)
except Exception as e:
    import traceback
    traceback.print_exc()

Epoch 1/5


2/4 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 1475.3142

: 

In [1]:
def find_ref_minmax(data):
    assert data.shape[2]==12 and data.shape[1]==5 #60 channels
    minarr=np.zeros(12)
    maxarr=np.zeros(12)
    for channel in range(data.shape[2]):  # Assuming the third index is for channels
        # Extract the channel data
        channel_data = data[:, :, channel, :, :]
        
        # Compute the min and max
        minarr[channel] = np.min(channel_data)
        maxarr[channel] = np.max(channel_data)
    return minarr,maxarr

def normalize_channels(data,minarr,maxarr):
    # data.shape should be (numDays*framesPerDay, numIsoLvls, numChannels, height, width)
    assert data.shape[2]==12 and data.shape[1]==5 #60 channels
    # Initialize a new array to hold the normalized data
    normalized_data = np.zeros_like(data, dtype=np.float32)
    
    # Iterate over each channel to normalize
    for channel in range(data.shape[2]):  # Assuming the third index is for channels
        # Extract the channel data
        channel_data = data[:, :, channel, :, :]
        
        # OBTAIN the min and max
        min_val = minarr[channel]
        max_val = maxarr[channel]
        
        # Avoid division by zero in case all values in the channel are the same
        if max_val > min_val:
            # Normalize the channel
            normalized_data[:, :, channel, :, :] = (channel_data - min_val) / (max_val - min_val)
        else:
            # If max equals min, set the normalized data to zero or handle appropriately
            normalized_data[:, :, channel, :, :] = 0  # Or set it to a neutral value like 0.5 if more appropriate
    return normalized_data

def Bdx_to_Adx(bdx, startFrameinA):
    """ Converts an index in best track data to its corresponding index in inTensor. """
    return bdx * 3 + startFrameinA

def data_generator(inTensor, besttrack_ndarr, startFrameinA, endFrameinA, scope=12, forecast_horizon=6):
    """
    Generator function to yield individual training data samples and labels.
    """
    # Calculate the number of samples available for generation
    num_samples = len(besttrack_ndarr) - forecast_horizon // 3
    indices = np.arange(0, num_samples)
    np.random.shuffle(indices)  # Shuffle indices for random sampling
    
    while True:  # Loop indefinitely for continuous generation
        for idx in indices:
            # Calculate starting frame index for inTensor based on best track data index
            start_idx = Bdx_to_Adx(idx, startFrameinA) - scope
            end_idx = Bdx_to_Adx(idx, startFrameinA) + 1
            
            # Generate input frames from inTensor
            cnn_input = np.stack(inTensor[start_idx:end_idx])
            
            # Get corresponding best track input
            track_input = besttrack_ndarr[idx]
            
            # Prepare target output based on forecast horizon
            target = besttrack_ndarr[idx + forecast_horizon // 3][:4]
            
            if cnn_input is None or track_input is None or target is None:
                print("None found in generator output!")
                continue  # or handle it differently as needed
            
            # print("Yielding data:", cnn_input.shape, track_input.shape, target.shape)
            # Yield the individual data sample and target
            yield ((cnn_input, track_input), target)

In [2]:
import pygrib
import numpy as np
import pandas as pd

gribName="20151018_PATRICIA_1024"
cyc_name=str.upper("PATRICIA")
grbs = pygrib.open('.\\Data\\'+gribName+'.grib')
grb=grbs[1:]
grb.append(grbs[len(grb)+1])
del grbs

numChannels=12
gridShape=(221,381)
framesPerDay=24
isoLvls=[500,700,850,925,1000]
numIsoLvls=len(isoLvls)
numFrames=len(grb)
numDays=numFrames//(numChannels*framesPerDay*numIsoLvls)
#INDEX=Frame*5*12+IsoLvl*12+Channel (tot=12*5*24*numDays
assert numFrames%(numChannels*framesPerDay*numIsoLvls)==0 #Having integer number of days
assert grb[1].values.shape==gridShape #Check grid shape matches

arr=np.zeros(shape=(numDays*framesPerDay,numIsoLvls,numChannels,*gridShape),dtype="float32")
print(arr.shape)

def parseIndices(i):
    timeLoc=i//(numChannels*numIsoLvls)
    isoLoc=(i-timeLoc*(numChannels*numIsoLvls))//numChannels
    channelLoc=i-timeLoc*(numChannels*numIsoLvls)-isoLoc*numChannels
    return timeLoc,isoLoc,channelLoc

#INDEX=Frame*60+IsoLvl*12+Channel (tot=12*5*24*numDays)
for i,v in enumerate(grb):
    arr[*parseIndices(i)] = v.values.astype("float32")
del grb

minref,maxref=find_ref_minmax(arr)


import tensorflow as tf

output_signature = (
    (
        tf.TensorSpec(shape=(13, *gridShape, 60), dtype=tf.float32),  # CNN input shape
        tf.TensorSpec(shape=(6,), dtype=tf.float32)  # Best track input shape
    ),
    tf.TensorSpec(shape=(4,), dtype=tf.float32)  # Output target shape
)

(168, 5, 12, 221, 381)


In [3]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model

# Input shape considering each frame is now part of a sequence
input_layer = Input(shape=(13, *gridShape, 60))  # Adjust the gridShape and channels as necessary

# Apply CNN to each frame independently using TimeDistributed
cnn_out = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))(input_layer)
cnn_out = TimeDistributed(MaxPooling2D(pool_size=(3, 3)))(cnn_out)
cnn_out = TimeDistributed(Conv2D(filters=64, kernel_size=(4, 4), activation='relu'))(cnn_out)
cnn_out = TimeDistributed(MaxPooling2D(pool_size=(3, 3)))(cnn_out)
cnn_out = TimeDistributed(Conv2D(filters=128, kernel_size=(4, 4), activation='relu'))(cnn_out)
cnn_out = TimeDistributed(MaxPooling2D(pool_size=(3, 3)))(cnn_out)
cnn_out = TimeDistributed(Flatten())(cnn_out)  # Flatten each frame's feature map

# If you want to aggregate features across time, you can use Flatten directly or a pooling layer across the time dimension
# Flatten to prepare for Dense layers (e.g., merging time and feature maps into one long vector)
cnn_out = Flatten()(cnn_out)

# Scalar inputs for the best track data at present (shape=(6,))
best_track_input = Input(shape=(6,))

# Concatenate the scalar values with the output of the CNN
combined_inputs = Concatenate(axis=-1)([cnn_out, best_track_input])

# Dense layers for final prediction
dense_out = Dense(units=100, activation='relu')(combined_inputs)
output = Dense(units=4)(dense_out)  # Predicting 4 values for the cyclone's next location

# Compile the model
model = Model(inputs=[input_layer, best_track_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 13, 221,   │          0 │ -                 │
│ (InputLayer)        │ 381, 60)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 13, 219,   │     17,312 │ input_layer[0][0] │
│ (TimeDistributed)   │ 379, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 13, 73,    │          0 │ time_distributed… │
│ (TimeDistributed)   │ 126, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, 13, 70,    │     32,832 │ time_distributed… │
│ (TimeDistributed)   │ 123, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 13, 23,    │          0 │ time_distributed… │
│ (TimeDistributed)   │ 41, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_4  │ (None, 13, 20,    │    131,200 │ time_distributed… │
│ (TimeDistributed)   │ 38, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_5  │ (None, 13, 6, 12, │          0 │ time_distributed… │
│ (TimeDistributed)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_6  │ (None, 13, 9216)  │          0 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 119808)    │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 119814)    │          0 │ flatten_1[0][0],  │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 100)       │ 11,981,500 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 4)         │        404 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,163,248 (46.40 MB)

 Trainable params: 12,163,248 (46.40 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
import datetime
def file_to_arr(fname):
    global minref,maxref
    global numFrames,numDays

    # cyc_name=str.upper(fname.split('_')[1])
    grbs = pygrib.open('.\\Data\\'+fname+'.grib')

    grb=grbs[1:]
    grb.append(grbs[len(grb)+1])

    firstDay=grb[0]['day']
    assert fname.split('_')[0][-2:]==str(firstDay)
    
    numFrames=len(grb)
    numDays=numFrames//(numChannels*framesPerDay*numIsoLvls)
    #INDEX=Frame*5*12+IsoLvl*12+Channel (tot=12*5*24*numDays)
    assert numFrames%(numChannels*framesPerDay*numIsoLvls)==0 #Having integer number of days
    assert grb[1].values.shape==gridShape #Check grid shape matches
    
    numFrames=len(grb)
    numDays=numFrames//(numChannels*framesPerDay*numIsoLvls)

    for i,v in enumerate(grb):
        arr[*parseIndices(i)] = v.values.astype("float32")

    del grb
    del grbs
    
    
    normalized_arr = normalize_channels(arr,minref,maxref)
    transposed_arr = np.transpose(normalized_arr, (0, 3, 4, 1, 2))

    # Now, we need to merge the last two dimensions (numIsoLvls and numChannels)
    # Calculate the new dimension
    new_last_dim = transposed_arr.shape[3] * transposed_arr.shape[4]

    # Reshape to merge the last two dimensions
    inTensor = transposed_arr.reshape(transposed_arr.shape[0], transposed_arr.shape[1], transposed_arr.shape[2], new_last_dim)

    return inTensor, numDays

def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

def load_btrack(fname="BestTrack_Trimmed.csv"):
    best_Track=pd.read_csv(fname)
    best_Track['ISO_TIME']=pd.to_datetime(best_Track['ISO_TIME'])
    fBesttrack=best_Track[(best_Track['ISO_TIME']>=pd.to_datetime('20150101',format='%Y%m%d'))&(best_Track['ISO_TIME']<=pd.to_datetime('20240101',format='%Y%m%d'))&(best_Track['LAT']>=0.0)&(best_Track['LAT']<=90.0)&(best_Track['LON']>=-170.0)&(best_Track['LON']<=-80.0)]
    selected_columns = fBesttrack[['USA_WIND', 'USA_PRES', 'LAT', 'LON','ISO_TIME','NAME']]
    selected_columns=selected_columns[selected_columns['ISO_TIME'].dt.hour%3==0]

    badrows=selected_columns[(selected_columns['USA_WIND']==' ') | (selected_columns['USA_PRES']==' ')]
    print("Null WP values removed")  
    print(badrows)
    selected_columns=selected_columns[(selected_columns['USA_WIND']!=' ')&(selected_columns['USA_PRES']!=' ')]

    # Define the ranges
    lat_min, lat_max = 0, 40
    lon_min, lon_max = -170, -80
    wind_min, wind_max = 20, 200
    pressure_min, pressure_max = 860, 1020
    selected_columns['LAT'] = normalize(selected_columns['LAT'], lat_min, lat_max)
    selected_columns['LON'] = normalize(selected_columns['LON'], lon_min, lon_max)
    selected_columns['USA_WIND'] = normalize(selected_columns['USA_WIND'].astype(float), wind_min, wind_max)
    selected_columns['USA_PRES'] = normalize(selected_columns['USA_PRES'].astype(float), pressure_min, pressure_max)
    selected_columns['ISO_DATE'] = selected_columns['ISO_TIME'].dt.dayofyear/365
    selected_columns['ISO_HOUR'] = selected_columns['ISO_TIME'].dt.hour/24
    return selected_columns

btrack=load_btrack()

def load_grib(*fname):
    assert len(fname)==1 or len(fname)==2
    cyc_name=str.upper(fname[0].split('_')[1])
    inTensor=None
    numdays=0

    if(len(fname)==2):
        intensorA,numDays=file_to_arr(fname[0])
        intensorB,day_concat=file_to_arr(fname[1])
        inTensor = np.concatenate((intensorA,intensorB),axis=0)
        numDays+=day_concat
    else: 
        inTensor,numDays=file_to_arr(fname[0])
    
    global btrack
    starttime=pd.to_datetime(fname[0][:8],format='%Y%m%d')
    endtime=starttime+pd.Timedelta(days=numDays)
    print(btrack.head(5))
    best_Track=btrack[(btrack['ISO_TIME']>=starttime)&(btrack['ISO_TIME']<=endtime)&(btrack['NAME']==cyc_name)]

    starttimeb=best_Track['ISO_TIME'].iloc[0]
    endtimeb=best_Track['ISO_TIME'].iloc[-1]
    startFrameinA=int((starttimeb-starttime)/datetime.timedelta(hours=1))
    endFrameinA=int((endtimeb-starttime)/datetime.timedelta(hours=1))

    assert (endtimeb-starttimeb)/datetime.timedelta(seconds=3*3600) == len(best_Track)-1 #same frame count
    besttrack_ndarr=best_Track[['USA_WIND', 'USA_PRES', 'LAT', 'LON','ISO_DATE','ISO_HOUR']].to_numpy().astype("float32")

    global dataset
    dataset = tf.data.Dataset.from_generator(
        generator=lambda: data_generator(inTensor, besttrack_ndarr, startFrameinA, endFrameinA),
        output_signature=output_signature
    ).batch(3)  # Define batch size here

Null WP values removed
      USA_WIND USA_PRES      LAT       LON            ISO_TIME    NAME
12522       15           12.3000  -98.2000 2023-06-25 06:00:00  ADRIAN
12523       15           12.4125  -98.2900 2023-06-25 09:00:00  ADRIAN
12524       15           12.5000  -98.4000 2023-06-25 12:00:00  ADRIAN
12525       15           12.5500  -98.5425 2023-06-25 15:00:00  ADRIAN
12526       15           12.6000  -98.7000 2023-06-25 18:00:00  ADRIAN
...        ...      ...      ...       ...                 ...     ...
13740       25           10.5500 -114.5530 2023-11-17 21:00:00   RAMON
13741       25           10.5000 -114.7000 2023-11-18 00:00:00   RAMON
13742       25           10.4500 -114.8420 2023-11-18 03:00:00   RAMON
13743       25           10.4000 -115.0000 2023-11-18 06:00:00   RAMON
13744       25           10.3500 -115.1930 2023-11-18 09:00:00   RAMON

[64 rows x 6 columns]


In [42]:
load_grib("20151018_PATRICIA_1024")

     USA_WIND  USA_PRES       LAT       LON            ISO_TIME    NAME  \
143  0.055556   0.90625  0.270000  0.667778 2015-05-28 06:00:00  ANDRES   
144  0.083333   0.89375  0.273690  0.661756 2015-05-28 09:00:00  ANDRES   
145  0.111111   0.88750  0.277500  0.655556 2015-05-28 12:00:00  ANDRES   
146  0.138889   0.87500  0.281438  0.649200 2015-05-28 15:00:00  ANDRES   
147  0.166667   0.86250  0.285000  0.643333 2015-05-28 18:00:00  ANDRES   

     ISO_DATE  ISO_HOUR  
143  0.405479     0.250  
144  0.405479     0.375  
145  0.405479     0.500  
146  0.405479     0.625  
147  0.405479     0.750  


In [43]:
model.fit(x=dataset, steps_per_epoch=10, epochs=4)

Epoch 1/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - loss: 28.8410
Epoch 2/4
10/10 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - loss: 0.0780
Epoch 3/4
 3/10 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - loss: 0.0674

ResourceExhaustedError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
MemoryError: Unable to allocate 251. MiB for an array with shape (13, 221, 381, 60) and data type float32
Traceback (most recent call last):

  File "c:\Users\KelM\anaconda3\envs\ecmwf2\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "c:\Users\KelM\anaconda3\envs\ecmwf2\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "c:\Users\KelM\anaconda3\envs\ecmwf2\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\KelM\AppData\Local\Temp\ipykernel_4376\3364562540.py", line 58, in data_generator